In [ ]:
pip install foolbox

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import numpy as np
import foolbox as fb
import time
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) #cifar100
])

# Normalize the test set same as training set without augmentation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) #cifar100
])

In [ ]:
batch_size = 128

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


In [ ]:
import torch.nn.functional as F

In [ ]:
def conv_block(in_channels, out_channels, pool=False):
    convo = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
    layers = [convo]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

In [ ]:
dt = 1 # standard ResNET

class ResNet(nn.Module):
    def __init__(self, in_chan, nf, n_layers):
        super(ResNet, self).__init__()

        self.nlayers = n_layers
        self.nf = nf
        self.chans = in_chan

        self.matrices = self.nlayers

        self.conv1 = conv_block(self.chans, self.nf,pool=False)
        self.convs = nn.ModuleList([nn.Conv2d(self.nf, self.nf,3,1,1,bias=True) for i in range(self.matrices)])
        self.convsO = nn.ModuleList([nn.Conv2d(self.nf, self.nf,3,1,1,bias=False) for i in range(self.matrices)])

        self.mp = nn.MaxPool2d(2,2)

    def forward(self, x):
        x = self.conv1(x)

        count = 0

        for i in np.arange(0,self.matrices):
          A = self.convs[i]
          B = self.convsO[i]
          x = x + dt * B(torch.relu(((A(x)))))

        x = self.mp(x)
        return x

class Network2(nn.Module):
    def __init__(self, in_chan, nf1, nf2, nf3, n_l1, n_l2, n_l3):
        super(Network2, self).__init__()

        self.input = in_chan
        self.nf1 = nf1
        self.nf2 = nf2
        self.nf3 = nf3
        self.n_l1 = n_l1
        self.n_l2 = n_l2
        self.n_l3 = n_l3

        self.seq = nn.Sequential(
            ResNet(self.input,self.nf1,self.n_l1),
            ResNet(self.nf1,self.nf2,self.n_l2),
            ResNet(self.nf2,self.nf3,self.n_l3),
            nn.Flatten(),
            nn.Linear(2048,100)
          )
 
    def forward(self,x):
      x = self.seq(x)
      return x

In [ ]:
lr = 1e-2
momentum = 0.9
optimizer = 'stochastic gradient descent'
scheduler = 'stepLR'
step_size = 30
EPOCHS = 100
weight_decay = 0

In [ ]:
class multiClassHingeLoss(nn.Module):
    def __init__(self, p=1, margin=1, device='cpu', size_average=True):
        super(multiClassHingeLoss, self).__init__()
        self.margin=margin
        self.size_average=size_average
        self.device = device
    def forward(self, output, y):
        output_y=output[torch.arange(0,y.size()[0]).long().to(self.device),y.data.to(self.device)].view(-1,1) #it is a (Batch Size x 1) tensor, having entries that are x[y]
        loss=output-output_y+self.margin #this has self.margin in position y and the difference between the entry of x and x[y] in the other positions
        #remove i=y items
        loss[torch.arange(0,y.size()[0]).long().to(self.device),y.data.to(self.device)]=0 #sets to 0 the entry in position y, instead of having self.margin
        #max(0,_)
        loss[loss<0]=0 #sets to 0 the entries of loss where we have negative numbers, i.e. those meeting the margin (there is a higher difference than the margin between x[y] and x[i])
        #sum up
        loss=torch.sum(loss)
        if(self.size_average):
            loss/=output.size()[0]
        return loss

In [ ]:
class Normalisation(torch.nn.Module):
    def __init__(self, means=(0.485, 0.456, 0.406), stds=(0.229, 0.224, 0.225)): #cifar100 
        super().__init__()
        assert len(means) == len(stds)
        self.means = means
        self.stds = stds
    
    def forward(self, x):
        return (x - torch.tensor(self.means, device=x.device).view(1, len(self.means), 1, 1)) / torch.tensor(self.stds, device=x.device).view(1, len(self.means), 1, 1)

In [ ]:
import torch.optim as optim

marginList = [0.07,0.15,0.3]
epsilons = [0.0, 8/255, 16/255, 36/255, 0.3, 0.5, 0.6, 0.8, 1.0]

robust_accuracy = np.zeros((len(marginList),len(epsilons)))

In [ ]:
for iterate,margin in enumerate(marginList):

  ResNet = Network2(3,32,64,128,4,4,4)
  ResNet.to(device);

  pretrained_dict = torch.load(f"cif10_trained_model_margin_{margin}.pt",map_location=device)
  ResNet.load_state_dict(pretrained_dict, strict=False)

  criterion = multiClassHingeLoss(margin = margin)
  optimizer = optim.SGD(ResNet.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

  EPOCHS = 100
  for epoch in range(EPOCHS):
      losses = []
      running_loss = 0
      correct = 0
      count = 0
      for i, inp in enumerate(trainloader):
          inputs, labels = inp
          inputs, labels = inputs.to(device), labels.to(device)
          optimizer.zero_grad()
      
          outputs = ResNet(inputs)
          loss = criterion(outputs, labels)
          losses.append(loss.item())
          loss.backward()

          optimizer.step()
          running_loss += loss.item()

          with torch.no_grad():
            _, predicted = torch.max(outputs.data, 1)
            count += 1
            correct += (predicted == labels.to(device)).sum().item() / len(inputs) #percentage of correct ones
          
          if i%100 == 0 and i > 0:
              print(f'Loss [{epoch+1}, {i}](epoch, minibatch): ', running_loss / 100)
              running_loss = 0.0
      
      #Check current accuracy
      correct = 0
      total = 0
      
      ResNet.eval()
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              images.shape
              # calculate outputs by running images through the network
              outputs = ResNet(images.to(device))
              # the class with the highest energy is what we choose as prediction
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels.to(device)).sum().item()
          print('Current accuracy on 10000 test images: %d %%' % (
              100 * correct / total))

      ResNet.train()
      scheduler.step()
      if epoch%10 == 0 and epoch>0:
          lr=optimizer.param_groups[0]["lr"]
  print('Training Done')

  transform_test_rob = transforms.Compose([
    transforms.ToTensor()
  ])
  batch_size = 1024
  testset_rob = torchvision.datasets.CIFAR100(root='./data', train=False,
                                        download=True, transform=transform_test_rob)
  testloader_rob = torch.utils.data.DataLoader(testset_rob, batch_size=batch_size,
                                          shuffle=False, num_workers=2)
  images, labels = next(iter(testloader_rob))
  images, labels = images.to(device), labels.to(device)
  model = nn.Sequential(Normalisation(),ResNet).eval()
  fmodel = fb.PyTorchModel(model, bounds=(0, 1))
  
  acc = fb.utils.accuracy(fmodel, images, labels)
  attack = fb.attacks.L2PGD(steps=10)
  
  _, advs, success = attack(fmodel, images, labels, epsilons=epsilons)
  robust_accuracy[iterate] = torch.mean((1-1.*success),axis=1).detach().cpu().numpy()

  fig = plt.figure(figsize=(20,10))
  plt.plot(epsilons,robust_accuracy[iterate],'r-*',label="Experimental")
  plt.xlabel(r"$\varepsilon$",fontsize=20)
  plt.ylabel("Robust accuracy",fontsize=20)
  plt.xticks(fontsize=20)
  plt.yticks(fontsize=20)
  plt.legend(fontsize=20,loc=1)
  plt.title(f"L2 robustness. Trained with margin = {margin}. Test accuracy = {round(acc * 100,2)}%",fontsize=20);

  plt.savefig(f'Cifar100_L2margin_{margin}.png')

  dataForTxt = robust_accuracy[iterate]
  destination = f"Cifar100_updateMargin_{margin}.txt"
  np.savetxt(destination, dataForTxt.reshape(-1,1))

  torch.save(ResNet.state_dict(), f"cif100_trained_model_margin_{margin}.pt")